In [ ]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from mlxtend.frequent_patterns import apriori,association_rules

df_train = pd.read_csv("/kaggle/input/network-intrusion-detection/Train_data.csv")

df_train.info()

In [ ]:
df_train.head(n=10)

In [ ]:
data_train = pd.get_dummies(df_train,columns = ["protocol_type", "flag", "service", "class"],)

data_train.head()


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# extract numerical attributes and scale it to have zero mean and unit variance  
cols = data_train.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(data_train.select_dtypes(include=['float64','int64']))


# turn the result back to a dataframe
sc_traindf = pd.DataFrame(sc_train, columns = cols)
sc_traindf.head(n=10)


In [ ]:
def encode_units(x):
    if x<=0:
        return 0
    if x>=0 :
        return 1
    
train_df = sc_traindf.applymap(encode_units)

train_df.head(n=10)

    

In [ ]:
train_df.info()

In [ ]:
columns = data_train.columns
colname = ['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']
for col in columns :
    for j in colname :
        if col == j :
            data_train[col] = train_df[col]
data_train.head()

In [ ]:
data_train[['num_outbound_cmds','is_host_login']].head()
for col in data_train.columns :
    if data_train[col].mean() == 0 :
        print(col)

In [ ]:
data_train.drop(["num_outbound_cmds", "is_host_login"], axis = 1, inplace=True)
data_train.info()


In [ ]:
frequent_itemsets = apriori ( data_train , min_support = 0.01 , use_colnames=True ,max_len =2)
result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc


In [ ]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.90)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

In [ ]:
rules[rules['consequents'] == {'class_anomaly'}]

In [ ]:
frequent_itemsets = apriori ( data_train , min_support = 0.33 , use_colnames=True ,max_len =2)
result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc


In [ ]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.75)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

In [ ]:
rules[rules['consequents'] == {'class_normal'}]